In [1]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from BiasedErasure.delayed_erasure_decoders.Experimental_Loss_Decoder import *
from BiasedErasure.main_code.Simulator import *
from BiasedErasure.main_code.noise_channels import atom_array
from BiasedErasure.delayed_erasure_decoders.HeraldedCircuit_SWAP_LD import HeraldedCircuit_SWAP_LD


In [2]:
def simulate(noise_params={}, chunk_size=10000, noise_scales=[1.,1.,1.,1.]):
    cycles = 4
    decoder_basis = 'Z'
    num_rounds = 5
    gate_ordering = ['N', 'Z', 'Zr', 'Nr']
    Meta_params = {'architecture': 'CBQC', 'code': 'Rotated_Surface', 'logical_basis': decoder_basis, 'bias_preserving_gates': 'False', 
        'noise': 'atom_array', 'is_erasure_biased': 'False', 'LD_freq': '100', 'LD_method': 'FREE', 'SSR': 'True', 'cycles': str(num_rounds - 1),
            'ordering': gate_ordering,
            'decoder': 'ML',
        'circuit_type': 'memory', 'Steane_type': 'regular', 'printing': 'False', 'num_logicals': '1', 'loss_decoder': 'independent', 'obs_pos': 'd-1', 'n_r': '0'}
    bloch_point_params = {'erasure_ratio': '1', 'bias_ratio': '0.5'}
    
    # Example data, change this:
    dx = 5
    dy = 5
    
    output_dir = ""
    simulator = Simulator(Meta_params=Meta_params, atom_array_sim=True, 
                                bloch_point_params=bloch_point_params, noise=atom_array , 
                                phys_err_vec=None, loss_detection_method=HeraldedCircuit_SWAP_LD, 
                                cycles = cycles, output_dir=output_dir, save_filename=None, save_data_during_sim=False)
    
    meas, det = simulator.sampling_with_loss(chunk_size, dx, dy, None, noise_params=noise_params)
    return torch.Tensor(meas).int(), torch.Tensor(det).int()

In [3]:
def p_ij_matrix(detections: np.ndarray) -> np.ndarray:
    num_ancillas = 24
    num_time_steps = 4
    total_steps = num_ancillas * num_time_steps
    
    p_ij = np.zeros((total_steps, total_steps))
    x_avg = np.mean(detections, axis=0)
    xixj_avg_matrix = np.dot(detections.T, detections) / detections.shape[0]
    for idx_i in range(total_steps):
        for idx_j in range(idx_i + 1, total_steps):
            xi_avg = x_avg[idx_i]
            xj_avg = x_avg[idx_j]
            xixj_avg = xixj_avg_matrix[idx_i, idx_j]
            numerator = 4 * (xixj_avg - xi_avg * xj_avg)
            denominator = 1 - 2 * xi_avg - 2 * xj_avg + 4 * xixj_avg
            if denominator > 0 and 1 - numerator / denominator >= 0:
                value = 0.5 - 0.5 * np.sqrt(1 - numerator / denominator)
                p_ij[idx_i, idx_j] = value
                p_ij[idx_j, idx_i] = value  # The matrix is symmetric
    return np.round(p_ij, 3)

In [4]:
from tqdm.auto import *
maximums = np.array([
        3e-6, *[8e-6]*3, *[0.05/4]*3, *[0.05/4]*15, 0.05/4, *[1e-3]*3, 0.03, 0.04 
    ])
to_dict = lambda noise_params: dict(
        idle_loss_rate=noise_params[0],
        idle_error_rate=noise_params[1:4],
        entangling_zone_error_rate=noise_params[4:7],
        entangling_gate_error_rate=noise_params[7:22],
        entangling_gate_loss_rate=noise_params[22],
        single_qubit_error_rate=noise_params[23:26],
        reset_error_rate=noise_params[26],
        measurement_error_rate=noise_params[27]
    )
keys = ['idle_loss_rate', *['idle_error_rate']*3, *['entangling_zone_error_rate']*3, 
        *['entangling_gate_error_rate']*15, 'entangling_gate_loss_rate', *['single_qubit_error_rate']*3,
        'reset_error_rate', 'measurement_error_rate'
        ]
fig, axes = plt.subplots(4,7,figsize=(10,5))
axes = axes.flatten()
for i in trange(28):
    x = np.zeros(28)
    x[i] = maximums[i]/5
    meas, det = simulate(noise_params=to_dict(x), chunk_size=20000)
    cov_ = p_ij_matrix(det.numpy()).flatten()
    cov_ /= np.linalg.norm(cov_)
    tmp = int(np.sqrt(len(cov_)))
    axes[i].imshow(cov_.reshape((tmp,tmp)))
    axes[i].axis('off')
    axes[i].set_title(f'{keys[i]}', fontsize=5)

fig.tight_layout()

  0%|          | 0/28 [00:00<?, ?it/s]

entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
entangling Pauli error rate = None, entangling loss rate = None
